In [12]:
import numpy as np
import pandas as pd
import os
import sys

from IPython.display import display
from scipy.stats import pearsonr
from scipy.cluster import hierarchy as hac

from random import randint
from scipy.stats import halfnorm
from scipy.spatial.distance import cosine

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [13]:
cc_dir = '../../2001/output/cross_corr/'
gam_ac_dir = '../../2001/output/auto_gam/'


In [14]:
''' iniitialize arrays'''

object_arr = np.arange(0,763)
num_obj = len(object_arr)

corr_matrix = np.empty([num_obj,num_obj])
corr_diff_matrix = np.empty([num_obj,num_obj])

corr_matrix.fill(np.nan)
corr_diff_matrix.fill(np.nan)

## sample one object to get array lengths
obj1 = 0
cc_file1 = cc_dir+'object'+str(obj1).zfill(4)+'_stats000.dat'

read_in = np.loadtxt(cc_file1)
time = read_in[0]
time = time[::5]
arr_time = np.logical_and(time >= -1000, time <= 2000)
time = time[arr_time]
num_time = np.sum(arr_time)
t_weight = np.exp(-abs(time)/1000.)
t_weight_diff = np.exp(-abs(time[:-1])/1000.)

num_iterations = 20 ## number of cc points per time

## create cc points matrix
cor_multi = np.zeros([num_obj, num_iterations, num_time])


In [15]:
''' populate cor_multi array'''

for ii in range(0,num_obj):
	obj = object_arr[ii]
	
	try:
		cc_file = cc_dir+'object'+str(obj).zfill(4)+'_stats000.dat'
		read_in = np.loadtxt(cc_file)

		data = read_in[1:,::5]
	except:
		continue
		
	data = data[:,arr_time] ## get rid of times outside range
# 	data = data[:,::5] ## get rid of a bunch of punchs
	
	cor_mean = data[0]
	cor_err_d = cor_mean-data[1]
	cor_err_u = data[2]-cor_mean
	
	arr1 = np.logical_and(np.isfinite(cor_mean), cor_mean > -1.1) ## array of workable points
	arr1_not = np.logical_not(arr1)
	
	num_temp = np.sum(arr1)
	cor_mean_temp = cor_mean[arr1]
	cor_err_d_temp = cor_err_d[arr1]
	cor_err_u_temp = cor_err_u[arr1]
	
	for jj in range(0,num_iterations):
		cc_vector = np.zeros(num_temp)
		
		for kk in range(0,num_temp):
			if randint(0,1) == 0:
				cc_vector[kk] = cor_mean_temp[kk] - halfnorm.rvs(scale=cor_err_d_temp[kk])
			else:
				cc_vector[kk] = cor_mean_temp[kk] + halfnorm.rvs(scale=cor_err_u_temp[kk])
	
		cor_multi[ii,jj,arr1] = cc_vector
		cor_multi[ii,jj,arr1_not] = np.nan
	print(ii, end=' ')
	
cor_diff_multi = cor_multi[:,:,1:] - cor_multi[:,:,:-1]

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 118 119 120 121 122 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 28

In [16]:
print(cor_multi[0,0].max())

0.655134469810174


In [20]:
''' Calculate distance and CC matrices'''
fac = 10. ## factor in arctan transform

for ii in range(0,num_obj):
	arr1 = np.isfinite(cor_multi[ii,0])
# 	arr1_diff = np.isfinite(cor_diff_multi[ii,0])
	
	for jj in range(ii,num_obj):
		arr2 = np.isfinite(cor_multi[jj,0])
# 		arr2_diff = np.isfinite(cor_diff_multi[jj,0])
		
		arr_tot = np.logical_and(arr1, arr2)
# 		arr_tot_diff = np.logical_and(arr1_diff, arr2_diff)
		
		cc_iter_arr = np.empty(num_iterations**2)
		cc_iter_arr.fill(np.nan)
		
# 		cc_diff_iter_arr = np.empty(num_iterations**2)
# 		cc_diff_iter_arr.fill(np.nan)
		
		iter_index = 0
		for kk in range(0,num_iterations):
			x = cor_multi[ii,kk,arr_tot]
# 			x_diff = cor_diff_multi[ii,kk,arr_tot_diff]
			
			for mm in range(0,num_iterations):
				y = cor_multi[jj,mm,arr_tot]
# 				y_diff = cor_diff_multi[jj,mm,arr_tot_diff]
				
				WEIGHT = t_weight[arr_tot]
# 				WEIGHT_diff = t_weight_diff[arr_tot_diff]
				
				## correlation between objects
				cc_corr = cosine(x,y, w=WEIGHT)
				cc_iter_arr[iter_index] = cc_corr
				
				## correlation difference between objects
# 				cc_corr_diff, cc_pval_diff = cosine(x_diff,y_diff, w=WEIGHT_diff)
# 				cc_diff_iter_arr[iter_index] = cc_corr_diff
				
				iter_index+= 1
		
		corr_matrix[ii,jj] = np.nanmean(cc_iter_arr)
# 		corr_diff_matrix[ii,jj] = np.nanmean(cc_diff_iter_arr)
	print(ii, end=' ')
		
np.savetxt('xobject_corr_matrix_cos.dat', corr_matrix)
# np.savetxt('xobject_corr_diff_matrix_cos.dat', corr_diff_matrix)

C:\Users\psyko\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
# ''' LOAD FROM FILES IF NOT CALCULATED BEFORE '''

# corr_matrix = np.loadtxt('xobject_corr_matrix_cos.dat')
# corr_diff_matrix = np.loadtxt('xobject_corr_diff_matrix_cos.dat')

In [ ]:
# print(corr_matrix)
# print(corr_diff_matrix)